In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Chapter 2 - More BeautifulSoup - Getting all the information from one movie

## Summmary of this notebook:
TODO:
We'll pick one movie from our list, the harry potter movie, and see how we can extract all the relevant information from that page.


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
base_url = 'http://www.boxofficemojo.com'
movie_url = 'movies/?id=harrypotter.htm'

In [ ]:
response = requests.get(url)

# Dont forget to adjust for 2017 terms
